In [1]:
%%capture
%run "./3-User-Experience-Analysis.ipynb"

In [2]:
display(user_experiance.head())
display(user_engagment.head())

,MSISDN/Number,avg_RTT_ms,avg_TCPr_bytes,avg_tp_kbps,clusters_by_experiance
0,33601001722.0,46.00,21777860.54,38.00,0
1,33601001754.0,31.00,21777860.54,49.50,0
2,33601002511.0,50.00,21777860.54,48.50,0
3,33601007832.0,84.00,772117.81,124.00,0
4,33601008617.0,119.00,31148692.54,28422.00,0


,MSISDN,session_freq,session_duration,session_data_volume,clusters_by_engagement
0,33614892860.0,17,9966905758.00,16341524035.00,1
1,33626320676.0,18,8791936680.00,15996644092.00,1
2,33625779332.0,17,18553763486.00,14701273330.00,1
3,33675877202.0,15,4865953827.00,14556184322.00,2
4,33659725664.0,16,4035436251.00,13888413360.00,2


In [6]:
engagement_model.cluster_centers_

array([[1.09391128e-09, 7.61361382e-02, 9.96417984e-01],
       [2.63129530e-09, 4.95174539e-01, 8.54597884e-01],
       [1.53747728e-09, 2.11636918e-01, 9.75714410e-01]])

In [5]:
experiance_model.cluster_centers_

array([[5.04010144e-04, 9.99673340e-01, 6.48644746e-03],
       [5.25655652e-03, 3.76632818e-01, 9.02661339e-01],
       [5.68409939e-03, 8.84204487e-01, 4.35449459e-01]])